
#### Author: Madhusudhanan Balasubramanian (MB), Ph.D., The University of Memphis
#### V11_Phase3-B copy of ResNet50_Model_V11_Phase3-A
Folder: 

In [ ]:
#Dec 10, 2021: based on train_axon_annotation_model.ipynb
import os
import sys
import time
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import imgaug.augmenters as imgAugmenters

# Root directory of the project
#ROOT_DIR = os.path.abspath("../../")
ROOT_DIR = "./Mask_RCNN";

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

#MB, Jan 06, 2022: Good configuration references
# https://medium.com/@umdfirecoml/training-a-mask-r-cnn-model-using-the-nucleus-data-bcb5fdbc0181
# https://medium.com/analytics-vidhya/taming-the-hyper-parameters-of-mask-rcnn-3742cb3f0e1b
# https://github.com/matterport/Mask_RCNN/issues/1884
#
#Model configuration for training
#---------------------------------
import axon_coco as coco #copied samples/coco/coco.py as axon_coco.py

class TrainingConfig(coco.CocoConfig):
    #Dec 09, 2021 MB notes: Initially, no other configuration changes needed for training (recall / see that
    # configuration changes required for inferences such as setting # GPUs to 1, etc. See axon_coco.py for 
    # other possible configuration changes
    
    #General model parameters
    BACKBONE = 'resnet50' #default is resnet101
    #BACKBONE_STRIDES = [2, 4, 8, 16, 32, 64]
    USE_MINI_MASK=True
    MEAN_PIXEL = [123.7, 116.8, 103.9]
        
    #General training parameters
    #----------------
    IMAGES_PER_GPU = 3 #should be 1 for inference
    GPU_COUNT = 1
    BATCH_SIZE = IMAGES_PER_GPU * GPU_COUNT #BATCH_SIZE calculated only in config.py's constructor in line 216
    #
    STEPS_PER_EPOCH = 10 #Jan 07: reduced from 100 to 75; May 10, 2022: steps/epoch = #training images/batch size
    VALIDATION_STEPS = 5 #originally 5, previously set at 15
    #
    GRADIENT_CLIP_NORM = 10.0
    LEARNING_MOMENTUM = 0.7
    WEIGHT_DECAY=0.0001
    LOSS_WEIGHTS = { "rpn_class_loss": 1., "rpn_bbox_loss": 1., "mrcnn_class_loss": 1., "mrcnn_bbox_loss": 1., "mrcnn_mask_loss": 1. }

    #RPN parameters
    #---------------
    # Length of square anchor side in pixels
    #RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    #RPN_ANCHOR_SCALES = (4, 8, 16, 32, 64) #Incorrectly set this in V11_Phase2 - that's why the model had issues
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128) #Was used in V11 which generated the model e608
    # Ratios of anchors at each cell (width/height)
    # A value of 1 represents a square anchor, and 0.5 is a wide anchor
    #RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    RPN_ANCHOR_RATIOS = [0.25, 0.5, 1, 2, 4]
    #References for increasing number of detections: https://github.com/matterport/Mask_RCNN/issues/1884#:~:text=What%20seems%20to%20have%20had%20the%20greatest%20impact%20for%20us%20were%20the%20training%20configs%3A
    RPN_TRAIN_ANCHORS_PER_IMAGE = 600 #default: 256; Number of anchors per image selected to train the RPN
    MAX_GT_INSTANCES = 600 #Number of GT instances per image kept to train the network
    
    #Proposal layer parameters (not trainable layer; just filtering)
    #-------------------------
    PRE_NMS_LIMIT = 6000 #default: ;Number of anchors with the best RPN score that are retained
    RPN_NMS_THRESHOLD = 0.7 #0.7 #default is 0.7; area overlap among candidate anchors before dropping the anchor with the lowest score; higher values increases the number of region proposals
    POST_NMS_ROIS_TRAINING = 1800 # >Training; ROIs kept after non-maximum supression in the proposal layer; default is 2000
    POST_NMS_ROIS_INFERENCE = 8000 # >Inference; ROIs kept after NMS in the proposal layer based on their RPN score
    
    #Detection target layer parameters (training only; not a trainable layer; just filtering)
    #Receives at most POST_NMS_ROIS_TRAINING number of anchors from the proposal layer
    #Anchors whose IoU > 0.5 over ground truth are selected (at most POST_NMS_ROIS_TRAINING)
    #---------------------------------
    TRAIN_ROIS_PER_IMAGE = 600 #default is 200; no. of ROIs randomly selected out of POST_NMS_ROIS_TRAINING
    ROI_POSITIVE_RATIO = 0.33 #default is 0.33; usually set as 1/#classes; i.e. 33% of TRAIN_ROIS_PER_IMAGE should be positive
       
    #Detection layer parameters (inference only; not trainable, just filtering)
    #Receives at most POST_NMS_ROIS_INFERENCE number of anchors from the proposal layer
    #MB: https://medium.com/@umdfirecoml/training-a-mask-r-cnn-model-using-the-nucleus-data-bcb5fdbc0181 
    #----------------------------
    DETECTION_MIN_CONFIDENCE = 0.7#default: 0.7; AOIs with lower confidence than this are discarded
    DETECTION_NMS_THRESHOLD = 0.3 #reference: 0.3; AOIs those with higher overlapping areas are discarded based on their RPN score

    #Feature Pyramid Network (FPN)
    #Has a classifier and mask graph; identifies class and generates mask
    #---------------------------------
    DETECTION_MAX_INSTANCES = 600 # >Inference; maximum number of instances identified by Mask RCNN
    
class InferenceConfig(TrainingConfig):
    
    #General training parameters
    #----------------
    IMAGES_PER_GPU = 1 #should be 1 for inference
    GPU_COUNT = 1
    BATCH_SIZE = IMAGES_PER_GPU * GPU_COUNT #BATCH_SIZE calculated only in config.py's constructor in line 216
        
config = TrainingConfig()
config.display()

inference_config = InferenceConfig()
inference_config.display()

#Data
#-----
COCO_DIR = "./DataFiles/Phase3"
#
#Axon training data
dataset_train = coco.CocoDataset()
dataset_train.load_coco(COCO_DIR, "Phase3_Training")
dataset_train.prepare() # Must call before using the dataset
#
#Axon annotation validation data
dataset_val = coco.CocoDataset()
dataset_val.load_coco(COCO_DIR, "Phase3_Validation")
dataset_val.prepare() # Must call before using the dataset

# Create a model in the "training" mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir = coco.DEFAULT_LOGS_DIR)

#Create a Mask RCNN model in "inference" mode
model_inference = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir = coco.DEFAULT_LOGS_DIR)

#Initialize the training model with pretrained weights
#Note: inference model weights are loaded in the modellib._MeanF1Callback() from the latest trained model
init_with = "specific"

if init_with == "imagenet":
    print("Loading ImageNet weights")
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    #MB, Jan 11, 2022
    # Download COCO trained weights from Releases if needed -- MB - this needs to be checked if it can download
    #if not os.path.exists(coco.COCO_MODEL_PATH):
    #    utils.download_trained_weights(COCO_MODEL_PATH)
    
    model_path = os.path.join(coco.ROOT_DIR, "mask_rcnn_coco.h5")
    #
    print("Loading weights from", model_path)
    model.load_weights(model_path, by_name = True,
                      exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                              "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "specific":
    #MB Dec 09, 2021: "specific" was added to resume training the axon model (not from an initial COCO model)
    # So, no need to exclude certain layers as above for "coco".  You would choose "coco" to start from scratch
    #m_dir = "/home/madhu/Lab/Members/00_madhu/Programs/axon_segmentation/logs/"
    #m_sdir = "coco20220327T1856"
    #m_name = "mask_rcnn_coco_0608.h5"
    #
    m_dir = "./"
    m_sdir = ""
    m_name = "axon_model_v11_e0608.h5"
    model_path = os.path.join(m_dir, m_sdir, m_name)
    #
    print("Loading weights from", model_path)
    model.load_weights(model_path, by_name=True)
elif init_with == "last":
    #Load the last model training to resume training
    model_path = model.find_last()
    #
    print("Loading weights from", model_path)
    model.load_weights(model_path, by_name=True)
    
# Image Augmentation
# Right/Left flip 50% of the time
# augmentation = imgaug.augmenters.Fliplr(0.5)
#max_augs = 4
#augmentation = imgAugmenters.SomeOf((0, max_augs),
#                                   [
#                                       imgAugmenters.Affine(scale={"x": (0.5, 1.25), "y": (0.5, 1.25)}),
#                                       imgAugmenters.Affine(rotate=(-45,45)),
#                                       imgAugmenters.Flipud(0.5),
#                                       imgAugmenters.Fliplr(0.5)
#                                       #imgAugmenters.GaussianBlur(sigma=(0.0, 8.0))
#                                   ])

#Call back function for tracking multiclass metrics
mean_f1_callback = modellib.MeanF1Callback(model, model_inference, dataset_val, 
                                           calculate_metrics_at_every_X_epochs=10, verbose=1)



Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     3
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        600
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             10.0
IMAGES_PER_GPU                 3
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.7
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from /home/madhu/Lab/Members/00_madhu/Models/axon_annotation/axon_model_v11_e0608.h5








# ARVO 2023

In [2]:
#Heads
config.LEARNING_MOMENTUM = 0.7
config.LEARNING_RATE = 0.00001
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=50, 
            layers='heads'#,
            #custom_callbacks=[mean_f1_callback]
           )

#Fine-tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=75,
            layers='all'#,
            #custom_callbacks=[mean_f1_callback]
           )


Starting at epoch 0. LR=1e-05. LM=0.7.

Checkpoint Path: /home/madhu/Lab/Members/00_madhu/Programs/axon_segmentation/logs/coco20230416T2151/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_ma

/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/50
10/10 [==============================] - 272s 27s/step - loss: 1.9135 - rpn_class_loss: 0.1397 - rpn_bbox_loss: 0.3576 - mrcnn_class_loss: 0.5654 - mrcnn_bbox_loss: 0.1991 - mrcnn_mask_loss: 0.6516 - val_loss: 2.2323 - val_rpn_class_loss: 0.2989 - val_rpn_bbox_loss: 0.4085 - val_mrcnn_class_loss: 0.6056 - val_mrcnn_bbox_loss: 0.2029 - val_mrcnn_mask_loss: 0.7164

Epoch 2/50
10/10 [==============================] - 15s 1s/step - loss: 1.8953 - rpn_class_loss: 0.1377 - rpn_bbox_loss: 0.3568 - mrcnn_class_loss: 0.5643 - mrcnn_bbox_loss: 0.1995 - mrcnn_mask_loss: 0.6369 - val_loss: 2.1839 - val_rpn_class_loss: 0.2959 - val_rpn_bbox_loss: 0.4078 - val_mrcnn_class_loss: 0.5838 - val_mrcnn_bbox_loss: 0.1987 - val_mrcnn_mask_loss: 0.6977
Epoch 3/50
10/10 [==============================] - 16s 2s/step - loss: 2.1050 - rpn_class_loss: 0.3327 - rpn_bbox_loss: 0.3979 - mrcnn_class_loss: 0.5311 - mrcnn_bbox_loss: 0.1992 - mrcnn_mask_loss: 0.6441 - val_loss: 2.1505 - val_rpn_class_loss:

10/10 [==============================] - 70s 7s/step - loss: 1.9705 - rpn_class_loss: 0.2181 - rpn_bbox_loss: 0.3874 - mrcnn_class_loss: 0.6068 - mrcnn_bbox_loss: 0.1934 - mrcnn_mask_loss: 0.5648 - val_loss: 1.8856 - val_rpn_class_loss: 0.2548 - val_rpn_bbox_loss: 0.3696 - val_mrcnn_class_loss: 0.5628 - val_mrcnn_bbox_loss: 0.1763 - val_mrcnn_mask_loss: 0.5220
Epoch 22/50
10/10 [==============================] - 49s 5s/step - loss: 1.8041 - rpn_class_loss: 0.1644 - rpn_bbox_loss: 0.3725 - mrcnn_class_loss: 0.5297 - mrcnn_bbox_loss: 0.1888 - mrcnn_mask_loss: 0.5486 - val_loss: 1.8312 - val_rpn_class_loss: 0.2522 - val_rpn_bbox_loss: 0.3691 - val_mrcnn_class_loss: 0.5089 - val_mrcnn_bbox_loss: 0.1778 - val_mrcnn_mask_loss: 0.5231
Epoch 23/50
10/10 [==============================] - 14s 1s/step - loss: 1.5886 - rpn_class_loss: 0.0876 - rpn_bbox_loss: 0.3505 - mrcnn_class_loss: 0.4482 - mrcnn_bbox_loss: 0.1847 - mrcnn_mask_loss: 0.5176 - val_loss: 1.8870 - val_rpn_class_loss: 0.2506 - val_

Epoch 43/50
10/10 [==============================] - 15s 1s/step - loss: 1.5602 - rpn_class_loss: 0.0975 - rpn_bbox_loss: 0.3387 - mrcnn_class_loss: 0.4659 - mrcnn_bbox_loss: 0.1793 - mrcnn_mask_loss: 0.4787 - val_loss: 1.6063 - val_rpn_class_loss: 0.1512 - val_rpn_bbox_loss: 0.3367 - val_mrcnn_class_loss: 0.5541 - val_mrcnn_bbox_loss: 0.1553 - val_mrcnn_mask_loss: 0.4090
Epoch 44/50
10/10 [==============================] - 15s 1s/step - loss: 1.5046 - rpn_class_loss: 0.1019 - rpn_bbox_loss: 0.3343 - mrcnn_class_loss: 0.4352 - mrcnn_bbox_loss: 0.1721 - mrcnn_mask_loss: 0.4610 - val_loss: 2.1092 - val_rpn_class_loss: 0.4394 - val_rpn_bbox_loss: 0.4179 - val_mrcnn_class_loss: 0.5934 - val_mrcnn_bbox_loss: 0.1742 - val_mrcnn_mask_loss: 0.4842
Epoch 45/50
10/10 [==============================] - 88s 9s/step - loss: 1.6202 - rpn_class_loss: 0.1548 - rpn_bbox_loss: 0.3576 - mrcnn_class_loss: 0.4654 - mrcnn_bbox_loss: 0.1736 - mrcnn_mask_loss: 0.4688 - val_loss: 2.2135 - val_rpn_class_loss: 0

/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/an

Epoch 51/75
10/10 [==============================] - 476s 48s/step - loss: 1.4976 - rpn_class_loss: 0.0695 - rpn_bbox_loss: 0.3275 - mrcnn_class_loss: 0.5074 - mrcnn_bbox_loss: 0.1683 - mrcnn_mask_loss: 0.4249 - val_loss: 1.7052 - val_rpn_class_loss: 0.1624 - val_rpn_bbox_loss: 0.3764 - val_mrcnn_class_loss: 0.5165 - val_mrcnn_bbox_loss: 0.1687 - val_mrcnn_mask_loss: 0.4811
Epoch 52/75
10/10 [==============================] - 19s 2s/step - loss: 1.4875 - rpn_class_loss: 0.0655 - rpn_bbox_loss: 0.3258 - mrcnn_class_loss: 0.5064 - mrcnn_bbox_loss: 0.1677 - mrcnn_mask_loss: 0.4220 - val_loss: 1.6869 - val_rpn_class_loss: 0.1580 - val_rpn_bbox_loss: 0.3754 - val_mrcnn_class_loss: 0.5187 - val_mrcnn_bbox_loss: 0.1628 - val_mrcnn_mask_loss: 0.4721
Epoch 53/75
10/10 [==============================] - 17s 2s/step - loss: 1.6404 - rpn_class_loss: 0.1995 - rpn_bbox_loss: 0.3669 - mrcnn_class_loss: 0.4721 - mrcnn_bbox_loss: 0.1664 - mrcnn_mask_loss: 0.4355 - val_loss: 1.6702 - val_rpn_class_loss:

Epoch 73/75
10/10 [==============================] - 26s 3s/step - loss: 1.2862 - rpn_class_loss: 0.0375 - rpn_bbox_loss: 0.3214 - mrcnn_class_loss: 0.3767 - mrcnn_bbox_loss: 0.1638 - mrcnn_mask_loss: 0.3869 - val_loss: 1.4784 - val_rpn_class_loss: 0.1395 - val_rpn_bbox_loss: 0.3440 - val_mrcnn_class_loss: 0.4640 - val_mrcnn_bbox_loss: 0.1513 - val_mrcnn_mask_loss: 0.3795
Epoch 74/75
10/10 [==============================] - 137s 14s/step - loss: 1.2955 - rpn_class_loss: 0.0429 - rpn_bbox_loss: 0.3229 - mrcnn_class_loss: 0.3810 - mrcnn_bbox_loss: 0.1617 - mrcnn_mask_loss: 0.3870 - val_loss: 1.4866 - val_rpn_class_loss: 0.1386 - val_rpn_bbox_loss: 0.3438 - val_mrcnn_class_loss: 0.4669 - val_mrcnn_bbox_loss: 0.1539 - val_mrcnn_mask_loss: 0.3834
Epoch 75/75
10/10 [==============================] - 51s 5s/step - loss: 1.3865 - rpn_class_loss: 0.0913 - rpn_bbox_loss: 0.3424 - mrcnn_class_loss: 0.3955 - mrcnn_bbox_loss: 0.1491 - mrcnn_mask_loss: 0.4082 - val_loss: 1.8373 - val_rpn_class_loss:

# All Training Stages

In [ ]:
# Train the head branches
config.LEARNING_MOMENTUM = 0.9
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE*10, 
            epochs=200, 
            layers='heads',
            custom_callbacks=[mean_f1_callback]
           )
            #augmentation=augmentation) # unfreeze head and just train on last layer

#Fine-tune Stage 4+
print("Fine tune Resnet stage 4 and up")
config.LEARNING_MOMENTUM = 0.9
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE*5,
            epochs=300,
            layers='4+',
            custom_callbacks=[mean_f1_callback]
           )
            #augmentation=augmentation)
    
#Fine-tune all layers
print("Fine tune all layers")
config.LEARNING_MOMENTUM = 0.9
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE*3,
            epochs=400,
            layers='all',
            custom_callbacks=[mean_f1_callback]
           )
            #augmentation=augmentation)

## Train heads

In [2]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
# RegEx: "heads": r"(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)"
#start_train = time.time()
heads_training_epochs = 100
config.LEARNING_MOMENTUM = 0.7
config.LEARNING_RATE = 0.0001
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=heads_training_epochs, 
            layers='heads'#,
            #custom_callbacks=[mean_f1_callback]
           )
            #augmentation=augmentation) # unfreeze head and just train on last layer
#end_train = time.time()
#minutes = round((end_train - start_train) / 60, 2)
#print(f'Heads training for {heads_training_epochs} epochs took {minutes} minutes')

#Save temporary model
#MB, Jan 11, 2022
#tmp_model_path = os.path.join(coco.ROOT_DIR, "on_axon_mrcnn_r50_heads_E12.h5")
#model.keras_model.save_weights(tmp_model_path)
#model.keras_model.save_weights(model_path)


Starting at epoch 0. LR=0.0001. LM=0.7.

Checkpoint Path: /home/madhu/Lab/Members/00_madhu/Programs/axon_segmentation/logs/coco20230416T1607/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_m

/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/100
10/10 [==============================] - 286s 29s/step - loss: 2.2212 - rpn_class_loss: 0.4036 - rpn_bbox_loss: 0.4155 - mrcnn_class_loss: 0.5410 - mrcnn_bbox_loss: 0.1977 - mrcnn_mask_loss: 0.6634 - val_loss: 2.6292 - val_rpn_class_loss: 0.6186 - val_rpn_bbox_loss: 0.4455 - val_mrcnn_class_loss: 0.6836 - val_mrcnn_bbox_loss: 0.2135 - val_mrcnn_mask_loss: 0.6679

Epoch 2/100
10/10 [==============================] - 17s 2s/step - loss: 2.0669 - rpn_class_loss: 0.3445 - rpn_bbox_loss: 0.4050 - mrcnn_class_loss: 0.5238 - mrcnn_bbox_loss: 0.1914 - mrcnn_mask_loss: 0.6021 - val_loss: 2.4580 - val_rpn_class_loss: 0.5386 - val_rpn_bbox_loss: 0.4374 - val_mrcnn_class_loss: 0.6660 - val_mrcnn_bbox_loss: 0.2045 - val_mrcnn_mask_loss: 0.6115
Epoch 3/100
10/10 [==============================] - 19s 2s/step - loss: 1.9431 - rpn_class_loss: 0.2248 - rpn_bbox_loss: 0.4013 - mrcnn_class_loss: 0.5610 - mrcnn_bbox_loss: 0.1892 - mrcnn_mask_loss: 0.5668 - val_loss: 2.3529 - val_rpn_class_lo

Epoch 43/100
10/10 [==============================] - 48s 5s/step - loss: 1.1513 - rpn_class_loss: 0.0234 - rpn_bbox_loss: 0.2980 - mrcnn_class_loss: 0.3602 - mrcnn_bbox_loss: 0.1522 - mrcnn_mask_loss: 0.3175 - val_loss: 1.4732 - val_rpn_class_loss: 0.1199 - val_rpn_bbox_loss: 0.3958 - val_mrcnn_class_loss: 0.4680 - val_mrcnn_bbox_loss: 0.1499 - val_mrcnn_mask_loss: 0.3396
Epoch 44/100
10/10 [==============================] - 32s 3s/step - loss: 1.0873 - rpn_class_loss: 0.0271 - rpn_bbox_loss: 0.3088 - mrcnn_class_loss: 0.3082 - mrcnn_bbox_loss: 0.1462 - mrcnn_mask_loss: 0.2970 - val_loss: 1.4018 - val_rpn_class_loss: 0.1082 - val_rpn_bbox_loss: 0.3920 - val_mrcnn_class_loss: 0.4212 - val_mrcnn_bbox_loss: 0.1529 - val_mrcnn_mask_loss: 0.3274
Epoch 45/100
10/10 [==============================] - 34s 3s/step - loss: 1.0678 - rpn_class_loss: 0.0276 - rpn_bbox_loss: 0.3104 - mrcnn_class_loss: 0.2896 - mrcnn_bbox_loss: 0.1454 - mrcnn_mask_loss: 0.2948 - val_loss: 1.3632 - val_rpn_class_loss

Epoch 65/100
10/10 [==============================] - 35s 4s/step - loss: 0.9847 - rpn_class_loss: 0.0289 - rpn_bbox_loss: 0.2739 - mrcnn_class_loss: 0.2974 - mrcnn_bbox_loss: 0.1259 - mrcnn_mask_loss: 0.2585 - val_loss: 1.4945 - val_rpn_class_loss: 0.1423 - val_rpn_bbox_loss: 0.4078 - val_mrcnn_class_loss: 0.4507 - val_mrcnn_bbox_loss: 0.1556 - val_mrcnn_mask_loss: 0.3380
Epoch 66/100
10/10 [==============================] - 31s 3s/step - loss: 1.2368 - rpn_class_loss: 0.0540 - rpn_bbox_loss: 0.3313 - mrcnn_class_loss: 0.4141 - mrcnn_bbox_loss: 0.1457 - mrcnn_mask_loss: 0.2917 - val_loss: 1.5399 - val_rpn_class_loss: 0.1401 - val_rpn_bbox_loss: 0.4075 - val_mrcnn_class_loss: 0.4931 - val_mrcnn_bbox_loss: 0.1600 - val_mrcnn_mask_loss: 0.3391
Epoch 67/100
10/10 [==============================] - 25s 2s/step - loss: 1.2468 - rpn_class_loss: 0.0557 - rpn_bbox_loss: 0.3385 - mrcnn_class_loss: 0.4131 - mrcnn_bbox_loss: 0.1462 - mrcnn_mask_loss: 0.2933 - val_loss: 1.4670 - val_rpn_class_loss

Epoch 87/100
10/10 [==============================] - 52s 5s/step - loss: 1.0430 - rpn_class_loss: 0.0310 - rpn_bbox_loss: 0.3001 - mrcnn_class_loss: 0.3168 - mrcnn_bbox_loss: 0.1267 - mrcnn_mask_loss: 0.2683 - val_loss: 1.2365 - val_rpn_class_loss: 0.0666 - val_rpn_bbox_loss: 0.3551 - val_mrcnn_class_loss: 0.3834 - val_mrcnn_bbox_loss: 0.1367 - val_mrcnn_mask_loss: 0.2947
Epoch 88/100
10/10 [==============================] - 23s 2s/step - loss: 1.1278 - rpn_class_loss: 0.0327 - rpn_bbox_loss: 0.3170 - mrcnn_class_loss: 0.3539 - mrcnn_bbox_loss: 0.1372 - mrcnn_mask_loss: 0.2871 - val_loss: 1.2451 - val_rpn_class_loss: 0.0714 - val_rpn_bbox_loss: 0.3585 - val_mrcnn_class_loss: 0.3710 - val_mrcnn_bbox_loss: 0.1395 - val_mrcnn_mask_loss: 0.3046
Epoch 89/100
10/10 [==============================] - 96s 10s/step - loss: 1.1629 - rpn_class_loss: 0.0386 - rpn_bbox_loss: 0.3283 - mrcnn_class_loss: 0.3642 - mrcnn_bbox_loss: 0.1394 - mrcnn_mask_loss: 0.2923 - val_loss: 1.2889 - val_rpn_class_los

## Fine tune ResNet Stage 5 and up

In [ ]:
# Finetune layers from ResNet stage 5 and up
# RegEx: "5+": r"(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)" in model.py
print("Fine tune Resnet stage 5 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=55,
            layers='5+')

## Fine tune ResNet Stage 4 and up

In [ ]:
#Training notes: e56-e70 with LEARNING_RATE (55%--1% mAP improvement); 
#e71-80-105-130 with LEARNING_RATE * 2 (60%--5% improvement);
#e130-e150: LR * 4; LM = 0.6(60%--0% improvement); 
#e151-175-LR*4, LM=0.7 (63%--mAP increase 3%)
#e176-e185--LR*4, LM=0.8 (62% - mAP decrease 1%)
#e186-e199-- (same) (63%)
#e200-e219 -- (same) (steady around 63%)
#e220-e229 -- LR*4, LM=0.9 (64.49%)
#e230-e239 -- LR*6, LM=0.9 (67.98%)
#e240-e249 -- LR*8, LM=0.9 (67.24%)
#e250-e259 -- LR*8, LM=0.9 (68.24%)
#e260-e265-e269 -- LR*10, LM=0.9 (69.03%)
#To try later: exxx with LEARNING_RATE * 1 and LEARNING_MOMENTUM increased from 0.7 to 0.9
#
print("Fine tune Resnet stage 4 and up")
config.LEARNING_MOMENTUM = 0.9
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE*10,
            epochs=25,
            layers='4+',
            custom_callbacks=[mean_f1_callback]
           )
            #augmentation=augmentation)
        
#Saving after training "4+" layers
#MB, Jan 11, 2022
#tmp_model_path = os.path.join(coco.ROOT_DIR, "on_axon_mrcnn_r50_4up_E20.h5")
#model.keras_model.save_weights(tmp_model_path)
#model.keras_model.save_weights(model_path)

## Fine tune 3+ layers

In [ ]:
#e379-e389: lr=*30; lm=0.8 -- 71.45%
#e389-e405-419: lr=*30; lm=0.9 -- 73.11%
#Note:
# With #RPN_NMS_THRESHOLD = 0.9; increased from 0.7 to increase number of RPN proposals
#       DETECTION_MIN_CONFIDENCE = 0.5; decreased from 0.7 to increase the number of detections
#       DETECTION_NMS_THRESHOLD = 0.3
#e395: lr=*30, lm=0.9 -- 77.09% (e395); so keeping these inference parameters for further inspection
#e396-419: lr=*30, lm=0.9 -- 76.06%
#e420-425-439-449: lr=*100, lm=0.7 -- 75.1%-76.24%

print("Fine tune Resnet stage 3 and up")
config.LEARNING_MOMENTUM = 0.7
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE*100,
            epochs=449,
            layers='3+',
            custom_callbacks=[mean_f1_callback]
           )
            #augmentation=augmentation)

## 3+ RPN layers with updated INFERENCE PARAMTERS; specifically, RPN_NMS_THRESHOLD = 0.9; DETECTION_MIN_CONFIDENCE = 0.5; DETECTION_NMS_THRESHOLD = 0.3

In [ ]:
#e450-e459: lr=*3; lm=0.7 -- 74.78%
#e459-e479: lr*3; lm=0.9 -- 75.32% (e479)
print("Fine tune Resnet stage 3 and up")
config.LEARNING_MOMENTUM = 0.9
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE*3,
            epochs=479,
            layers='3+',
            custom_callbacks=[mean_f1_callback]
           )
            #augmentation=augmentation)

## Lowering DETECTION_MIN_CONFIDENCE from 0.7 to 0.5 gave mAP ~ 77%.  But visually, this causes double labeling (see COCO Annotator validation images -- in M6800 laptop).  So, resetting to 0.7 and beginning "all" training from e395; april 21: increasing further to 0.9 to reduce double labelling

In [3]:
#e395-e409(with 0.9 min_detection); lr*1, lm=0.7 - 73.7% (70.19%)
#e410-e419-429; lr*1, lm=0.8 - 74.78%-74.45%
#e430-e449-459(with 0.9 min_detection)-464; lr*5, lm=0.8 - 75.34%-75.13%(73.97%)-(73.89%)
#e465-e479 (with 0.9 min_detection henceforth); lr*10, lm=0.8 -- 73.87%
#e480-e489 (with 0.9 min_detection henceforth); lr*5, lm=0.9 -- 74.18% (e481)
#e481-e489-e499; lr*3, lm=0.9--74.23% (e485)
#e500-509-519-529; lr*5,8,8 lm=0.9--74.16 (e502),74.20% (e518), 73.99% (e528)
#e530-569; lr*10,lm=0.9--74.23% (e530) 74.69% (e566)
#Changing val to val_rev_1 henceforth (big axon label flipped); steps_per_epoch=9; validation_steps = 4
#e567-602; lr*10,lm=0.9--74.86(e589)
#e590-629; lr*10, lm=0.9 -- 75.13 (e607)
#USE_MINI_MASK=False
#e608-629: lr*20; lm=0.9
print("Fine tune all layers")
config.LEARNING_MOMENTUM = 0.7
config.LEARNING_RATE = 0.0001
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=125,
            layers='all'#,
            #custom_callbacks=[mean_f1_callback]
           )
            #augmentation=augmentation)

Fine tune all layers

Starting at epoch 100. LR=0.0001. LM=0.7.

Checkpoint Path: /home/madhu/Lab/Members/00_madhu/Programs/axon_segmentation/logs/coco20230416T1607/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branc

/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/envs/venv_mamba/envs/venv_maskrcnn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/an

Epoch 101/125
10/10 [==============================] - 342s 34s/step - loss: 1.0945 - rpn_class_loss: 0.0686 - rpn_bbox_loss: 0.3402 - mrcnn_class_loss: 0.2783 - mrcnn_bbox_loss: 0.1365 - mrcnn_mask_loss: 0.2710 - val_loss: 1.3758 - val_rpn_class_loss: 0.1356 - val_rpn_bbox_loss: 0.3766 - val_mrcnn_class_loss: 0.4197 - val_mrcnn_bbox_loss: 0.1502 - val_mrcnn_mask_loss: 0.2936
Epoch 102/125
10/10 [==============================] - 17s 2s/step - loss: 1.0698 - rpn_class_loss: 0.0596 - rpn_bbox_loss: 0.3344 - mrcnn_class_loss: 0.2726 - mrcnn_bbox_loss: 0.1358 - mrcnn_mask_loss: 0.2674 - val_loss: 1.3369 - val_rpn_class_loss: 0.1313 - val_rpn_bbox_loss: 0.3745 - val_mrcnn_class_loss: 0.3939 - val_mrcnn_bbox_loss: 0.1447 - val_mrcnn_mask_loss: 0.2925
Epoch 103/125
10/10 [==============================] - 29s 3s/step - loss: 1.1036 - rpn_class_loss: 0.0419 - rpn_bbox_loss: 0.3347 - mrcnn_class_loss: 0.3141 - mrcnn_bbox_loss: 0.1316 - mrcnn_mask_loss: 0.2812 - val_loss: 1.3425 - val_rpn_class

Epoch 123/125
10/10 [==============================] - 21s 2s/step - loss: 0.9297 - rpn_class_loss: 0.0229 - rpn_bbox_loss: 0.2946 - mrcnn_class_loss: 0.2204 - mrcnn_bbox_loss: 0.1331 - mrcnn_mask_loss: 0.2587 - val_loss: 1.0369 - val_rpn_class_loss: 0.0441 - val_rpn_bbox_loss: 0.2916 - val_mrcnn_class_loss: 0.3185 - val_mrcnn_bbox_loss: 0.1277 - val_mrcnn_mask_loss: 0.2550
Epoch 124/125
10/10 [==============================] - 39s 4s/step - loss: 0.9063 - rpn_class_loss: 0.0212 - rpn_bbox_loss: 0.2886 - mrcnn_class_loss: 0.2093 - mrcnn_bbox_loss: 0.1310 - mrcnn_mask_loss: 0.2563 - val_loss: 1.0271 - val_rpn_class_loss: 0.0442 - val_rpn_bbox_loss: 0.2914 - val_mrcnn_class_loss: 0.3183 - val_mrcnn_bbox_loss: 0.1235 - val_mrcnn_mask_loss: 0.2496
Epoch 125/125
10/10 [==============================] - 24s 2s/step - loss: 0.8018 - rpn_class_loss: 0.0189 - rpn_bbox_loss: 0.2393 - mrcnn_class_loss: 0.1840 - mrcnn_bbox_loss: 0.1150 - mrcnn_mask_loss: 0.2447 - val_loss: 1.1352 - val_rpn_class_l